## Gaussian Pyramid Levels

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
# !pip install online_triplet_loss

import os
import random

import cv2
import numpy as np
import matplotlib.pyplot as plt
from online_triplet_loss.losses import *
from PIL import Image
import torch
from torch.utils.data import Dataset
from torchvision import datasets, transforms
import torch.nn as nn
from torch.nn import functional as F
from torch import linalg
from scipy.spatial.distance import pdist
from sklearn.cluster import KMeans
from copy import deepcopy

from collections import defaultdict
from tqdm import tqdm



from torch.utils.data import DataLoader

import torch.optim as optim

from data import get_triplets

## Learning

In [3]:
class Backbone(nn.Module):
    def __init__(self, in_chan, out_dim, num_classes):
        super(Backbone, self).__init__()
        
        self.conv1 = nn.Conv2d(in_channels=in_chan, out_channels=16, kernel_size=3, stride=1)
        self.bn1 = nn.BatchNorm2d(16)
        self.relu1 = nn.ReLU()
        
        self.conv2 = nn.Conv2d(in_channels=16, out_channels=32, kernel_size=3, stride=1)
        self.bn2 = nn.BatchNorm2d(32)
        self.relu2 = nn.ReLU()
        
        self.conv3 = nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3, stride=1)
        self.bn3 = nn.BatchNorm2d(64)
        self.relu3 = nn.ReLU()
        
        self.conv4 = nn.Conv2d(in_channels=64, out_channels=128, kernel_size=3, stride=1, padding=1)
        self.bn4 = nn.BatchNorm2d(128)
        self.relu4 = nn.ReLU()
        self.maxpool4 = nn.MaxPool2d(kernel_size=2, stride=2)
        
        self.conv5 = nn.Conv2d(in_channels=128, out_channels=256, kernel_size=3, stride=1, padding=1)
        self.bn5 = nn.BatchNorm2d(256)
        self.relu5 = nn.ReLU()
        self.maxpool5 = nn.MaxPool2d(kernel_size=2, stride=2)        
        
        self.pooling = nn.AdaptiveAvgPool2d((1, 1))
        
        self.fc = nn.Linear(in_features=256, out_features=out_dim)
        self.relu6 = nn.ReLU()
        
        self.classifier = nn.Linear(out_dim, num_classes)
        
    def forward(self, x):
        x = self.relu1(self.bn1(self.conv1(x)))
        x = self.relu2(self.bn2(self.conv2(x)))
        x = self.relu3(self.bn3(self.conv3(x)))

        x = self.relu4(self.bn4(self.conv4(x)))
        x = self.maxpool4(x)
        
        x = self.relu5(self.bn5(self.conv5(x)))
        x = self.maxpool5(x)

        x = self.pooling(x)
        embed = self.fc(x.squeeze())
        
        x = self.relu6(embed)
        x = self.classifier(x)
        
        return x, embed

In [4]:
class PetDataset(Dataset):
    def __init__(self, flist, transform, labels):
        
        self.flist = flist
        self.transform = transform
        self.labels = np.array(labels).astype("int64")
        assert len(flist) == len(labels)

    def __len__(self):
        return len(self.flist)

    def __getitem__(self, index):
        sample = self.flist[index]

        # read in the image, apply the standard transformation
        img = self.transform(Image.open(sample))

        return img, self.labels[index]

In [5]:
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

In [6]:
# use the augmented images
src = "pet/train/output"
flist_full = [os.path.join(src, f) for f in sorted(os.listdir(src))]

In [7]:
"""
Steps in every epoch:

1. Create random labels, create a dataset, dataloader with these labels
2. Do a forward & backward pass, using the feature vectors perform k-means clustering
3. Use cluster assignments as the labels and redefine the dataset and dataloader
4. Go to 2

"""

'\nSteps in every epoch:\n\n1. Create random labels, create a dataset, dataloader with these labels\n2. Do a forward & backward pass, using the feature vectors perform k-means clustering\n3. Use cluster assignments as the labels and redefine the dataset and dataloader\n4. Go to 2\n\n'

In [8]:
NUM_CLASSES = 20
NUM_CLUSTERS = NUM_CLASSES

In [10]:
# cluster the features
def cluster_features(features, num_clusters=5):
    
    cobj = KMeans(n_clusters=num_clusters)
    cobj.fit(features)    
    assignments = cobj.labels_
    
    return assignments 


def get_sizes(labels):
    
    clusters = defaultdict(int)
    for l in labels:
        clusters[l] += 1
    
    return clusters

In [11]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
FEAT_DIM = 512

# declare the network
model = Backbone(in_chan=3, out_dim=FEAT_DIM, num_classes=NUM_CLASSES)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.005)
# print(model)

model = model.to(device)
model.train()

CLUSTER_EVERY = 500
BATCH_SIZE = 16
flist = random.sample(flist_full, CLUSTER_EVERY*BATCH_SIZE)

num_samples = len(flist)

# create random labels
labels = np.random.randint(low=0, high=20, size=num_samples)

# create the dataset and dataloader
NUM_WORKERS = 4
train_dataset = PetDataset(flist, transform, labels)
train_loader = DataLoader(train_dataset, batch_size = BATCH_SIZE, shuffle=True, num_workers=NUM_WORKERS)

running_loss = 0.0
num_epochs = 20
for epoch in range(num_epochs):
    
    # get embeddings
    embeds = None
    for idx, (x, y) in enumerate(tqdm(train_loader)):
        
        images_ = x.to(device)
        labels_ = y.to(device)
        
        # Zero the parameter gradients
        optimizer.zero_grad()

        # Forward pass
        outputs, batch_embeds = model(images_)
        
        # Accumulate the embeddings
        batch_embeds = batch_embeds.clone().detach().cpu().numpy()
        if embeds is None:
            embeds = batch_embeds.copy()
        else:
            embeds = np.concatenate([embeds, batch_embeds], axis=0)
            
        loss = criterion(outputs, labels_)

        # Backward pass and optimize
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        
        if (idx+1) % CLUSTER_EVERY == 0:
            break

    print(f'Epoch [{epoch + 1}/{num_epochs}], Loss: {running_loss / CLUSTER_EVERY:.4f}')
    running_loss = 0.0
        
    # perform the clustering
    labels = cluster_features(embeds, num_clusters=NUM_CLUSTERS)  
    
    cluster_log = get_sizes(labels)
    
    print(f"Size of clusters: {cluster_log}")
    
    # re-define the dataset
    train_dataset = PetDataset(flist, transform, labels)
    train_loader = DataLoader(train_dataset, batch_size = 16, shuffle=True, num_workers=NUM_WORKERS)

100%|█████████▉| 499/500 [01:21<00:00,  6.13it/s]


Epoch [1/20], Loss: 3.0030
Size of clusters: defaultdict(<class 'int'>, {11: 146, 17: 161, 19: 15, 13: 379, 16: 455, 2: 1877, 1: 48, 5: 202, 4: 104, 6: 1729, 15: 1124, 3: 48, 7: 101, 14: 209, 8: 349, 0: 682, 10: 169, 9: 130, 18: 37, 12: 35})


100%|█████████▉| 499/500 [01:20<00:00,  6.21it/s]


Epoch [2/20], Loss: 2.4623
Size of clusters: defaultdict(<class 'int'>, {13: 1065, 5: 765, 8: 171, 6: 1091, 14: 242, 7: 59, 1: 108, 2: 455, 10: 51, 3: 29, 17: 86, 12: 784, 15: 25, 16: 324, 18: 70, 9: 116, 11: 153, 4: 601, 19: 687, 0: 1118})


100%|█████████▉| 499/500 [01:20<00:00,  6.18it/s]


...

Epoch [18/20], Loss: 2.2622
Size of clusters: defaultdict(<class 'int'>, {17: 20, 5: 106, 1: 104, 13: 70, 10: 20, 16: 193, 11: 43, 8: 168, 15: 5, 2: 89, 19: 193, 14: 78, 12: 306, 7: 34, 4: 9, 9: 823, 6: 276, 3: 368, 0: 903, 18: 4192})


100%|█████████▉| 499/500 [01:19<00:00,  6.24it/s]


Epoch [19/20], Loss: 1.9366
Size of clusters: defaultdict(<class 'int'>, {14: 112, 1: 89, 17: 18, 13: 82, 4: 67, 11: 24, 16: 65, 8: 24, 3: 69, 19: 53, 15: 99, 7: 6, 5: 153, 18: 187, 12: 63, 6: 439, 2: 147, 9: 2952, 10: 695, 0: 2656})


100%|█████████▉| 499/500 [01:18<00:00,  6.36it/s]


Epoch [20/20], Loss: 1.8510
Size of clusters: defaultdict(<class 'int'>, {9: 96, 1: 80, 19: 48, 11: 80, 3: 80, 18: 96, 6: 128, 12: 160, 16: 400, 0: 800, 10: 496, 4: 384, 15: 304, 5: 576, 14: 449, 8: 799, 2: 929, 13: 560, 7: 960, 17: 575})


In [12]:
torch.save(model.state_dict(), "pet_data_dc_e20.pth")



